<a href="https://colab.research.google.com/github/Tseng0318/yolo_waste/blob/main/light_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

In [5]:
import yaml

In [ ]:
!ls /content/drive/MyDrive/new_classification/

data.yaml  README.dataset.txt  README.roboflow.txt  test  train  val


In [ ]:
dataset_path = "/content/drive/MyDrive/new_classification"
splits = ["train", "val", "test"]  # Change "valid" to "val" if needed

In [6]:
# Define the correct dataset root path
dataset_root = "/content/drive/MyDrive/new_classification"

In [7]:
# Define the yaml path
yaml_path = "/content/drive/MyDrive/new_classification/data.yaml"

In [ ]:
def verify():
  '''function to verify the path'''
  for split in splits:
      images_path = os.path.join(dataset_path, split, "images")
      labels_path = os.path.join(dataset_path, split, "labels")

      print(f"\nChecking {split}:")
      print(f"{len(os.listdir(images_path))} images found" if os.path.exists(images_path) else f"No 'images' folder in {split}!")
      print(f"{len(os.listdir(labels_path))} labels found" if os.path.exists(labels_path) else f"No 'labels' folder in {split}!")

In [ ]:
# Function to check and clean images & labels
def clean_labels_and_images():
   #'''function to clean and delete any missing labels and images ''''
    for split in splits:
        image_dir = os.path.join(dataset_path, split, "images")
        label_dir = os.path.join(dataset_path, split, "labels")

        if not os.path.exists(image_dir) or not os.path.exists(label_dir):
            print(f"Missing 'images' or 'labels' folder in {split}/")
            continue

        # List all image and label files (without extensions)
        images = {f.rsplit(".", 1)[0] for f in os.listdir(image_dir) if f.endswith((".jpg", ".png"))}
        labels = {f.rsplit(".", 1)[0] for f in os.listdir(label_dir) if f.endswith(".txt")}

        # Find mismatches
        images_without_labels = images - labels
        labels_without_images = labels - images

        print(f"\nChecking {split}/")
        print(f"Found {len(images)} images and {len(labels)} labels")

        # Delete images without labels
        if images_without_labels:
            print(f"Deleting {len(images_without_labels)} images without labels...")
            for img_name in images_without_labels:
                img_path = os.path.join(image_dir, img_name + ".jpg")
                png_path = os.path.join(image_dir, img_name + ".png")
                if os.path.exists(img_path):
                    os.remove(img_path)
                if os.path.exists(png_path):
                    os.remove(png_path)

        # Delete labels without images
        if labels_without_images:
            print(f"Deleting {len(labels_without_images)} labels without images...")
            for lbl_name in labels_without_images:
                lbl_path = os.path.join(label_dir, lbl_name + ".txt")
                if os.path.exists(lbl_path):
                    os.remove(lbl_path)

        print("Cleanup completed!")

In [ ]:
# Run the cleanup function
clean_labels_and_images()


Checking train/
Found 7324 images and 7334 labels
Deleting 10 labels without images...
Cleanup completed!

Checking val/
Found 2118 images and 2098 labels
Deleting 20 images without labels...
Cleanup completed!

Checking test/
Found 1042 images and 1042 labels
Cleanup completed!


In [ ]:
verify()


Checking train:
7324 images found
7324 labels found

Checking val:
2098 images found
2098 labels found

Checking test:
1042 images found
1042 labels found


In [ ]:
# Function to check data.yaml
def check_yaml():
    """Function to check the dataset.yaml file."""
    if not os.path.exists(yaml_path):
        print(f"ERROR: {yaml_path} not found!")
        return

    # Load YAML file
    with open(yaml_path, "r") as f:
        data = yaml.safe_load(f)

    print("\nChecking dataset.yaml...")

    # Check required keys
    required_keys = ["path", "train", "val", "test", "nc", "names"]
    for key in required_keys:
        if key not in data:
            print(f"Missing key in YAML: {key}")

    # Print YAML contents
    print("\n YAML File Contents:")
    for key, value in data.items():
        print(f"{key}: {value}")

    # Check if paths exist
    dataset_root = data.get("path", "")
    if not dataset_root:
        print("Dataset root path is missing!")
        return

    for split in ["train", "val", "test"]:
        split_path = os.path.join(dataset_root, data.get(split, ""))
        if not os.path.exists(split_path):
            print(f"ERROR: {split} path not found -> {split_path}")
        else:
            print(f" {split} path found -> {split_path}")

    print("\nYAML file check completed!")

In [ ]:
# Load the YAML file
yaml_path = f"{dataset_root}/data.yaml"

# Read the existing YAML file
with open(yaml_path, "r") as file:
    data = yaml.safe_load(file)

# Add the missing 'path' key if not present
if "path" not in data:
    data["path"] = dataset_root

# Update paths to absolute paths
data["train"] = f"{dataset_root}/train/images"
data["val"] = f"{dataset_root}/val/images"
data["test"] = f"{dataset_root}/test/images"

# Write the updated YAML file
with open(yaml_path, "w") as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"Updated {yaml_path} successfully!")

Updated /content/drive/MyDrive/new_classification/data.yaml successfully!


In [ ]:
check_yaml()


Checking dataset.yaml...

 YAML File Contents:
names: ['BIODEGRADABLE', 'CARDBOARD', 'GLASS', 'METAL', 'PAPER', 'PLASTIC']
nc: 6
path: /content/drive/MyDrive/new_classification
roboflow: {'license': 'CC BY 4.0', 'project': 'garbage-classification-3', 'url': 'https://universe.roboflow.com/material-identification/garbage-classification-3/dataset/2', 'version': 2, 'workspace': 'material-identification'}
test: /content/drive/MyDrive/new_classification/test/images
train: /content/drive/MyDrive/new_classification/train/images
val: /content/drive/MyDrive/new_classification/val/images
 train path found -> /content/drive/MyDrive/new_classification/train/images
 val path found -> /content/drive/MyDrive/new_classification/val/images
 test path found -> /content/drive/MyDrive/new_classification/test/images

YAML file check completed!


In [8]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.1/949.1 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [9]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Download the model
model = YOLO("yolov8n.pt")  # also try 'yolov8s.pt' or 'yolov8n.pt'

100%|██████████| 6.25M/6.25M [00:00<00:00, 383MB/s]


In [ ]:
model.train(
    data="/content/drive/MyDrive/new_classification/data.yaml",
    epochs=10,  # Train full model
    workers=4,  # Increase workers if GPU allows
    imgsz=640,
    batch=32,  # Adjust based on GPU capacity
    device="cuda",
    optimizer="AdamW",
    pretrained=False,  # Keep pretrained weights (important for moderate dataset)
    lr0=0.0005,  # Lower learning rate for stable fine-tuning
    lrf=0.05,  # Reduce final LR gradually
    weight_decay=0.0001,
    warmup_epochs=1.0,  # Short warmup since model is pre-trained
    warmup_momentum=0.8,
    patience=5,  # Early stopping if no improvement
    save=True,
    # save_period=5,  # Save model every 5 epochs

    # Adjusted Augmentations
    mosaic=0.0,  # Disable mosaic for classification
    degrees=90,  # Allow 90-degree rotations
    translate=0.1,  # Slight increase
    scale=0.1,  # Slight increase
    fliplr=0.5,  # Increase horizontal flipping
    flipud=0.5,  # Enable vertical flipping

    # Reduce HSV shifts
    hsv_h=0.005,
    hsv_s=0.05,
    hsv_v=0.05,

    name="garbage_detection",
    project="/content/drive/MyDrive/new_classification/runs"
)

Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/new_classification/data.yaml, epochs=10, time=None, patience=5, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=4, project=/content/drive/MyDrive/new_classification/runs, name=garbage_detection, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sa

100%|██████████| 755k/755k [00:00<00:00, 87.3MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 395MB/s]


AMP: checks passed ✅


...

      10/10      16.2G      1.336      1.374      1.598        142        640: 100%|██████████| 229/229 [00:43<00:00,  5.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:10<00:00,  3.21it/s]


                   all       2098      18916      0.493      0.379      0.414      0.252

10 epochs completed in 0.152 hours.
Optimizer stripped from /content/drive/MyDrive/new_classification/runs/garbage_detection/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/new_classification/runs/garbage_detection/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/new_classification/runs/garbage_detection/weights/best.pt...
Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:14<00:00,  2.22it/s]


                   all       2098      18916      0.494      0.379      0.414      0.252
         BIODEGRADABLE        676      13637      0.734      0.463      0.576      0.303
             CARDBOARD        438       1292      0.644      0.409      0.502       0.33
                 GLASS        805       2380      0.799      0.527      0.664      0.419
                 METAL        398       1360        0.5      0.518      0.501      0.306
                 PAPER         15         33     0.0184     0.0606     0.0447      0.044
               PLASTIC         91        214       0.27      0.294      0.198      0.107
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /content/drive/MyDrive/new_classification/runs/garbage_detection


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ef72b450e50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [23]:
model2 = YOLO('/content/drive/MyDrive/new_classification/runs/garbage_detection/weights/last.pt')  # Load latest checkpoint

model2.train(
    data="/content/drive/MyDrive/new_classification/data.yaml",
    epochs=100,  # Continue for 50+ more epochs
    workers=4,
    imgsz=640,
    batch=32,
    device="cuda",
    optimizer="AdamW",
    pretrained=False,  # Already trained, don't use base YOLOv8n.pt
    lr0=0.0003,  # Lower learning rate for fine-tuning
    lrf=0.01,  # Reduce final learning rate decay
    weight_decay=0.0001,
    warmup_epochs=0,  # No warmup needed since training is continuing
    patience=50,  # Allow early stopping if no improvement
    save=True,
    save_period=5,  # Continue saving every 5 epochs

    # Augmentation Tweaks to Improve Recall
    scale=0.2,  # Allow more scaling variation
    translate=0.2,  # Allow more position shifts
    fliplr=0.4,
    flipud=0.4,

    name="garbage_detection_continued",
    project="/content/drive/MyDrive/new_classification/runs"
)

Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/new_classification/runs/garbage_detection/weights/last.pt, data=/content/drive/MyDrive/new_classification/data.yaml, epochs=100, time=None, patience=50, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=4, project=/content/drive/MyDrive/new_classification/runs, name=garbage_detection_continued, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False

train: Scanning /content/drive/MyDrive/new_classification/train/labels.cache... 7324 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7324/7324 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/new_classification/val/labels.cache... 2098 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2098/2098 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/new_classification/runs/garbage_detection_continued/labels.jpg... 
optimizer: AdamW(lr=0.0003, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to /content/drive/MyDrive/new_classification/runs/garbage_detection_continued
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2098      18916      0.628      0.473      0.545      0.376



100 epochs completed in 1.534 hours.
Optimizer stripped from /content/drive/MyDrive/new_classification/runs/garbage_detection_continued/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/new_classification/runs/garbage_detection_continued/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/new_classification/runs/garbage_detection_continued/weights/best.pt...
Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:14<00:00,  2.23it/s]


                   all       2098      18916      0.619       0.48      0.551       0.38
         BIODEGRADABLE        676      13637      0.821      0.487      0.641      0.367
             CARDBOARD        438       1292      0.749      0.516      0.625      0.468
                 GLASS        805       2380      0.883      0.693      0.812        0.6
                 METAL        398       1360      0.758      0.629      0.696      0.474
                 PAPER         15         33      0.053     0.0909     0.0789     0.0707
               PLASTIC         91        214      0.451      0.463      0.454      0.303
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /content/drive/MyDrive/new_classification/runs/garbage_detection_continued


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ef7273b20d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [12]:
# Load latest checkpoint
model2 = YOLO('/content/drive/MyDrive/new_classification/runs/garbage_detection_continued/weights/last.pt')

In [15]:
# Train with improved settings
model2.train(
    data="/content/drive/MyDrive/new_classification/data.yaml",
    epochs=100,  # Continue training
    workers=4,
    imgsz=640,
    batch=32,
    device="cuda",
    optimizer="AdamW",
    pretrained=False,  # Already trained, don't use base YOLOv8n.pt

    # Learning Rate Adjustments
    lr0=0.00015,  # Reduce LR for fine-tuning stability
    lrf=0.005,  # Lower decay for better long-term learning
    cos_lr=True,  # Enable cosine decay for smoother convergence
    weight_decay=0.0001,
    warmup_epochs=0,  # No warmup needed since training is continuing

    # Early Stopping & Saving
    patience=20,  # Stop if no improvement for 20 epochs
    save=True,
    save_period=5,  # Continue saving every 5 epochs

    # Improved Augmentations
    scale=0.2,  # More scaling variation
    translate=0.2,  # Allow more position shifts
    fliplr=0.4,  # Keep moderate flipping
    flipud=0.4,
    copy_paste=0.5,  # Generate synthetic PAPER & PLASTIC samples
    mixup=0.1,  # Improve generalization with MixUp
    dropout = 0.1,
    # Recall Enhancements
    iou=0.45,  # Lower IoU threshold to increase recall
    agnostic_nms=True,  # Handle overlapping objects better

    # Weighted Loss for Rare Classes
    cls=0.7,  # Boost classification accuracy
    dfl=1.8,  # Improve bounding box refinement

    # Training Name
    name="garbage_detection_pause_continue",
    project="/content/drive/MyDrive/new_classification/runs"
)

Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/new_classification/runs/garbage_detection_continued/weights/last.pt, data=/content/drive/MyDrive/new_classification/data.yaml, epochs=100, time=None, patience=20, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=4, project=/content/drive/MyDrive/new_classification/runs, name=garbage_detection_pause_continue, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.45, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agn

train: Scanning /content/drive/MyDrive/new_classification/train/labels.cache... 7324 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7324/7324 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/new_classification/val/labels.cache... 2098 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2098/2098 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/new_classification/runs/garbage_detection_pause_continue/labels.jpg... 
optimizer: AdamW(lr=0.00015, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to /content/drive/MyDrive/new_classification/runs/garbage_detection_pause_continue
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2098      18916       0.64      0.497      0.556      0.376
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 64, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



84 epochs completed in 1.384 hours.
Optimizer stripped from /content/drive/MyDrive/new_classification/runs/garbage_detection_pause_continue/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/new_classification/runs/garbage_detection_pause_continue/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/new_classification/runs/garbage_detection_pause_continue/weights/best.pt...
Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:14<00:00,  2.34it/s]


                   all       2098      18916      0.617      0.505      0.556      0.378
         BIODEGRADABLE        676      13637      0.782      0.547      0.666      0.369
             CARDBOARD        438       1292      0.739      0.546      0.632      0.453
                 GLASS        805       2380      0.873      0.712      0.818      0.607
                 METAL        398       1360      0.783      0.646        0.7      0.476
                 PAPER         15         33     0.0627     0.0909     0.0817     0.0721
               PLASTIC         91        214      0.465      0.491      0.442      0.292
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /content/drive/MyDrive/new_classification/runs/garbage_detection_pause_continue


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bee9d65e110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [17]:
from ultralytics import YOLO

# Load the best previous model
model = YOLO("/content/drive/MyDrive/new_classification/runs/garbage_detection_pause_continue/weights/best.pt")

# Start training
model.train(
    data="/content/drive/MyDrive/new_classification/data.yaml",
    epochs=20,  # Extended for better convergence
    batch=32,
    imgsz=640,
    device="cuda",
    workers=4,
    optimizer="AdamW",
    pretrained=False,  # Keeps the best weights from before
    lr0=0.0001,  # Lower for fine-tuning stability
    lrf=0.002,  # More gradual decay
    weight_decay=0.00005,  # Prevents overfitting
    warmup_epochs=0,  # No warmup needed
    dropout=0.2,  # Prevent overfitting
    multi_scale=True,  # Train on multiple sizes for better generalization
    patience=30,  # Allow more time for improvements
    save=True,
    save_period=5,  # Save checkpoints every 5 epochs
    augment=True,  # Enable strong augmentations
    mosaic=1.0,  # Stronger Mosaic augmentation
    mixup=0.3,  # Introduce MixUp augmentation
    copy_paste=0.5,  # Use Copy-Paste augmentation
    auto_augment="randaugment",  # Enable automatic augmentation
    hsv_h=0.02, hsv_s=0.8, hsv_v=0.6,  # Stronger color augmentations
    translate=0.3, scale=0.3, flipud=0.5, fliplr=0.5,  # More aggressive augmentations
    iou=0.5,  # IoU threshold for training
    max_det=300,  # Keep high max detection count
    overlap_mask=True, mask_ratio=4,  # Keeps good mask overlap
    project="/content/drive/MyDrive/new_classification/runs",
    name="garbage_detection_final_train"
)

Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/new_classification/runs/garbage_detection_pause_continue/weights/best.pt, data=/content/drive/MyDrive/new_classification/data.yaml, epochs=20, time=None, patience=30, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=4, project=/content/drive/MyDrive/new_classification/runs, name=garbage_detection_final_train, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agno

train: Scanning /content/drive/MyDrive/new_classification/train/labels.cache... 7324 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7324/7324 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/new_classification/val/labels.cache... 2098 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2098/2098 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/new_classification/runs/garbage_detection_final_train/labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=5e-05), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to /content/drive/MyDrive/new_classification/runs/garbage_detection_final_train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2098      18916       0.61      0.506      0.557      0.375



20 epochs completed in 0.339 hours.
Optimizer stripped from /content/drive/MyDrive/new_classification/runs/garbage_detection_final_train/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/new_classification/runs/garbage_detection_final_train/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/new_classification/runs/garbage_detection_final_train/weights/best.pt...
Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:16<00:00,  1.97it/s]


                   all       2098      18916      0.621      0.509      0.559      0.381
         BIODEGRADABLE        676      13637      0.829      0.497      0.668      0.371
             CARDBOARD        438       1292      0.759      0.538      0.658      0.488
                 GLASS        805       2380      0.888      0.704      0.822      0.612
                 METAL        398       1360       0.77       0.63      0.698      0.477
                 PAPER         15         33     0.0672      0.121     0.0829     0.0714
               PLASTIC         91        214      0.416      0.565      0.428      0.269
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /content/drive/MyDrive/new_classification/runs/garbage_detection_final_train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bee98f7b310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     